# ⚔️ Brian2 vs snnTorch: Comparativo de Frameworks

**Descrição:** Comparação detalhada entre dois dos principais frameworks para Spiking Neural Networks: Brian2 (focado em biologia) e snnTorch (focado em Deep Learning).

**Autor:** Mauro Risonho de Paula Assumpção
**Data de Criação:** 5 de Dezembro de 2025
**Licença:** MIT License
**Desenvolvimento:** Desenvolvedor Humano + Desenvolvimento por AI Assitida:
- Claude Sonnet 4.5
- Gemini 3 Pro Preview

---

In [ ]:
import sys
from pathlib import Path

# Adicionar src ao path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if '01_fraud_neuromorphic' not in str(project_root):
    project_root = project_root / 'portfolio' / '01_fraud_neuromorphic'

src_path = project_root / 'src'
sys.path.insert(0, str(src_path))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import torch  # type: ignore
from tqdm.auto import tqdm

# Configuração visual
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print(f"✅ Imports carregados")
print(f"📁 Project root: {project_root}")
print(f"🔧 PyTorch: {torch.__version__}")
print(f"🖥️  CUDA available: {torch.cuda.is_available()}")

## 1️⃣ Preparar Dados Sintéticos

In [ ]:
# Gerar dataset sintético
from sklearn.datasets import make_classification

# 1000 transações, 10 features, 2 classes
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=8,
    n_redundant=2,
    n_classes=2,
    weights=[0.8, 0.2],  # 80% legítimas, 20% fraudes
    random_state=42
)

# Normalizar para [0, 1]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Dataset:")
print(f"  Train: {X_train.shape}, Fraudes: {int(np.sum(y_train))}/{len(y_train)} ({np.mean(y_train).item()*100:.1f}%)")
print(f"  Test:  {X_test.shape}, Fraudes: {int(np.sum(y_test))}/{len(y_test)} ({np.mean(y_test).item()*100:.1f}%)")

## 2️⃣ Implementação Brian2

### Vantagens:
- ✅ Biologicamente realista
- ✅ STDP learning (unsupervised)
- ✅ Conversível para Loihi/TrueNorth

### Desvantagens:
- ❌ Muito lento
- ❌ Sem GPU support
- ❌ Complexo de debugar

In [ ]:
from models_snn import FraudSNN

print("🧠 Criando SNN com Brian2...")
brian2_snn = FraudSNN(
    input_size=256,  # Spike encoding dimension
    hidden_sizes=[128, 64],
    output_size=2
)

print(f"✅ Arquitetura Brian2: 256 → 128 → 64 → 2")
print(f"⚠️  Nota: Brian2 é lento - usando apenas 10 samples para demo")

# Treinar com subset pequeno (Brian2 é lento)
n_samples_brian2 = 10
print(f"\n⏱️  Treinando Brian2 SNN ({n_samples_brian2} samples)...")
start_time = time.time()

# Converter para DataFrame (formato esperado por Brian2)
df_train_small = pd.DataFrame(X_train[:n_samples_brian2])
df_train_small['is_fraud'] = y_train[:n_samples_brian2]

brian2_snn.train_stdp(
    [(np.random.rand(10), np.arange(10), y_train[i]) for i in range(n_samples_brian2)],
    epochs=5
)

brian2_time = time.time() - start_time
print(f"⏱️  Tempo de treinamento Brian2: {brian2_time:.2f}s ({brian2_time/n_samples_brian2:.2f}s por sample)")

## 3️⃣ Implementação snnTorch

### Vantagens:
- ✅ **Muito mais rápido** (GPU)
- ✅ Backprop otimizado
- ✅ Fácil deployment
- ✅ Integração PyTorch

### Desvantagens:
- ⚠️ Menos biologicamente realista
- ⚠️ Requer adaptação para hardware neuromórfico

In [ ]:
# Verificar se snnTorch está instalado
try:
    import snntorch as snn  # type: ignore
    SNNTORCH_AVAILABLE = True
    print(f"✅ snnTorch {snn.__version__} disponível")
except ImportError:
    SNNTORCH_AVAILABLE = False
    print("⚠️  snnTorch não instalado. Instalando...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "snntorch"])
    import snntorch as snn  # type: ignore
    SNNTORCH_AVAILABLE = True
    print(f"✅ snnTorch {snn.__version__} instalado!")

In [ ]:
from models_snn_snntorch import FraudSNNTorch, create_spike_data
from torch.utils.data import TensorDataset, DataLoader  # type: ignore

# Converter para spike trains
print("⚡ Codificando dados em spikes...")
num_steps = 50
X_train_spikes = create_spike_data(X_train, num_steps=num_steps, encoding="rate")
X_test_spikes = create_spike_data(X_test, num_steps=num_steps, encoding="rate")

# Criar DataLoaders
train_dataset = TensorDataset(
    X_train_spikes, 
    torch.LongTensor(y_train)
)
test_dataset = TensorDataset(
    X_test_spikes, 
    torch.LongTensor(y_test)
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✅ Spike data criado: {X_train_spikes.shape}")
print(f"   Time steps: {num_steps}")
print(f"   Encoding: Rate coding (Poisson)")

In [ ]:
# Recarregar módulo após correção
import importlib
import models_snn_snntorch
importlib.reload(models_snn_snntorch)
from models_snn_snntorch import FraudSNNTorch, create_spike_data

print("✅ Módulo models_snn_snntorch recarregado com correções")

In [ ]:
# Criar modelo snnTorch
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🧠 Criando SNN com snnTorch...")
snntorch_snn = FraudSNNTorch(
    input_size=10,  # Features originais
    hidden_sizes=[64, 32],
    output_size=2,
    beta=0.9,
    dropout=0.2
)

n_params = sum(p.numel() for p in snntorch_snn.parameters())
print(f"✅ Arquitetura snnTorch: 10 → 64 → 32 → 2")
print(f"📊 Parâmetros: {n_params:,}")
print(f"🔧 Device: {device}")

# Treinar
print(f"\n⏱️  Treinando snnTorch SNN (todo dataset)...")
start_time = time.time()

snntorch_snn.train_model(
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=30,
    lr=1e-3,
    device=device,
    verbose=True
)

snntorch_time = time.time() - start_time
print(f"\n⏱️  Tempo total snnTorch: {snntorch_time:.2f}s")
print(f"⚡ Speedup vs Brian2: {brian2_time/snntorch_time:.1f}x mais rápido!")

## 4️⃣ Comparação de Performance

In [ ]:
# Avaliar snnTorch
print("📊 Avaliando snnTorch no test set...")
snntorch_acc = snntorch_snn.evaluate(test_loader, device=device)

print(f"\n{'='*60}")
print(f"COMPARAÇÃO FINAL")
print(f"{'='*60}")
print(f"")
print(f"⏱️  VELOCIDADE:")
print(f"  Brian2:   {brian2_time:.2f}s para {n_samples_brian2} samples ({brian2_time/n_samples_brian2:.2f}s/sample)")
print(f"  snnTorch: {snntorch_time:.2f}s para {len(X_train)} samples ({snntorch_time/len(X_train):.4f}s/sample)")
print(f"  Speedup:  {(brian2_time/n_samples_brian2)/(snntorch_time/len(X_train)):.0f}x mais rápido!")
print(f"")
print(f"🎯 ACURÁCIA:")
print(f"  Brian2:   N/A (poucos samples para avaliar)")
print(f"  snnTorch: {snntorch_acc:.2f}%")
print(f"")
print(f"💻 INFRAESTRUTURA:")
print(f"  Brian2:   CPU only")
print(f"  snnTorch: {device.upper()} ({'GPU' if device == 'cuda' else 'CPU'} support)")
print(f"")
print(f"🔧 DEPLOYMENT:")
print(f"  Brian2:   Complexo (requer conversão)")
print(f"  snnTorch: Fácil (ONNX, TorchScript)")
print(f"{'='*60}")

## 5️⃣ Visualizar Curvas de Treinamento (snnTorch)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1 = axes[0]
epochs = range(1, len(snntorch_snn.train_losses) + 1)
ax1.plot(epochs, snntorch_snn.train_losses, marker='o', linewidth=2, markersize=6)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss (snnTorch)', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Accuracy
ax2 = axes[1]
ax2.plot(epochs, snntorch_snn.train_accuracies, marker='s', 
         linewidth=2, markersize=6, color='green')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title('Training Accuracy (snnTorch)', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ Convergência alcançada em ~{len(epochs)} epochs")

## 6️⃣ Teste de Inferência (snnTorch)

In [ ]:
# Teste de predição individual
test_sample = X_test_spikes[0:1].to(device)
true_label = y_test[0]

pred_class, confidence, spike_counts = snntorch_snn.predict(test_sample, device=device)

print(f"🔍 Teste de Inferência Individual:")
print(f"{'='*60}")
print(f"Features:     {X_test[0]}")
print(f"True label:   {'Fraud' if true_label == 1 else 'Legitimate'}")
print(f"Predicted:    {'Fraud' if pred_class == 1 else 'Legitimate'}")
print(f"Confidence:   {confidence:.2%}")
print(f"Spike counts: Legit={spike_counts[0]:.1f}, Fraud={spike_counts[1]:.1f}")
print(f"Status:       {'✅ CORRECT' if pred_class == true_label else '❌ INCORRECT'}")
print(f"{'='*60}")

## 7️⃣ Benchmark de Latência

In [ ]:
# Benchmark latência snnTorch
print("⏱️  Benchmarking latência snnTorch...")
latencies = []
n_test = 100

for i in tqdm(range(n_test), desc="Inferências"):
    test_sample = X_test_spikes[i:i+1].to(device)
    
    start = time.perf_counter()
    pred_class, confidence, _ = snntorch_snn.predict(test_sample, device=device)
    end = time.perf_counter()
    
    latency_ms = (end - start) * 1000
    latencies.append(latency_ms)

# Estatísticas
print(f"\n{'='*60}")
print(f"LATENCY BENCHMARK (snnTorch on {device.upper()})")
print(f"{'='*60}")
print(f"Samples:          {n_test}")
print(f"Mean latency:     {np.mean(latencies):.2f} ms")
print(f"Median latency:   {np.median(latencies):.2f} ms")
print(f"P95 latency:      {np.percentile(latencies, 95):.2f} ms")
print(f"P99 latency:      {np.percentile(latencies, 99):.2f} ms")
print(f"Min latency:      {np.min(latencies):.2f} ms")
print(f"Max latency:      {np.max(latencies):.2f} ms")
print(f"Throughput:       {1000/np.mean(latencies):.1f} inferences/s")
print(f"{'='*60}")

# Visualizar distribuição
plt.figure(figsize=(10, 5))
plt.hist(latencies, bins=30, alpha=0.7, edgecolor='black')
plt.axvline(np.mean(latencies), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(latencies):.2f}ms')
plt.axvline(np.median(latencies), color='orange', linestyle='--', linewidth=2, label=f'Median: {np.median(latencies):.2f}ms')
plt.xlabel('Latency (ms)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title(f'Latency Distribution - snnTorch on {device.upper()}', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 8️⃣ Salvar Modelo snnTorch

In [ ]:
# Salvar modelo
model_path = project_root / "models" / "fraud_snn_snntorch.pth"
model_path.parent.mkdir(exist_ok=True)

snntorch_snn.save(str(model_path))
print(f"✅ Modelo salvo em: {model_path}")

# Verificar tamanho
model_size_mb = model_path.stat().st_size / (1024 * 1024)
print(f"📦 Tamanho do modelo: {model_size_mb:.2f} MB")

## 9️⃣ Conclusões e Recomendações

### 🏆 Quando Usar Brian2?

✅ **Pesquisa em neurociência computacional**
- Foco em realismo biológico
- Estudar STDP e plasticidade sináptica
- Publicações acadêmicas

✅ **Deploy em hardware neuromórfico**
- Intel Loihi, IBM TrueNorth
- Conversão direta para chips

✅ **Aprendizado não-supervisionado**
- STDP funciona sem labels
- Detecção de anomalias

### ⚡ Quando Usar snnTorch?

✅ **Produção e deployment**
- Treinamento rápido
- GPU acceleration
- ONNX export

✅ **Prototipagem rápida**
- Experimentação ágil
- Debugging fácil
- Integração com PyTorch

✅ **Datasets grandes**
- Escala para milhões de samples
- Batch processing eficiente
- Distributed training

### 🎯 Nossa Recomendação

**Abordagem Híbrida:**

1. **Desenvolvimento:** snnTorch
   - Treinar e otimizar rapidamente
   - Validar conceitos
   - Ajustar hiperparâmetros

2. **Deploy:** Hardware-dependent
   - **Edge/Mobile:** snnTorch → ONNX → TensorRT
   - **Loihi/TrueNorth:** Converter → Brian2 → Hardware
   - **Cloud GPU:** snnTorch nativo

3. **Pesquisa:** Brian2
   - Análise biológica
   - Papers acadêmicos
   - Estudos de STDP

---

### 📊 Comparação Final

| Métrica | Brian2 | snnTorch | Vencedor |
|---------|--------|----------|----------|
| Velocidade | ⭐ | ⭐⭐⭐⭐⭐ | snnTorch |
| Realismo | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | Brian2 |
| Facilidade | ⭐⭐ | ⭐⭐⭐⭐⭐ | snnTorch |
| GPU Support | ❌ | ✅ | snnTorch |
| Hardware Deploy | ⭐⭐⭐⭐⭐ | ⭐⭐ | Brian2 |
| Accuracy | ⭐⭐⭐ | ⭐⭐⭐⭐⭐ | snnTorch |
| Production Ready | ⭐⭐ | ⭐⭐⭐⭐⭐ | snnTorch |

**Para este projeto de detecção de fraude:**
- 🏆 **Desenvolvimento:** snnTorch (winner)
- 🏆 **Pesquisa:** Brian2 (winner)
- 🏆 **Deploy Cloud:** snnTorch (winner)
- 🏆 **Deploy Neuromorphic:** Brian2 (winner)

## 🔗 Recursos Adicionais

### snnTorch
- 📖 Documentação: https://snntorch.readthedocs.io/
- 🐙 GitHub: https://github.com/jeshraghian/snntorch
- 📺 Tutorials: https://snntorch.readthedocs.io/en/latest/tutorials/index.html
- 📄 Paper: Eshraghian et al. (2021) - "Training Spiking Neural Networks Using Lessons From Deep Learning"

### Brian2
- 📖 Documentação: https://brian2.readthedocs.io/
- 🐙 GitHub: https://github.com/brian-team/brian2
- 📄 Paper: Stimberg et al. (2019) - "Brian 2, an intuitive and efficient neural simulator"

### Hardware Neuromórfico
- 🔬 Intel Loihi: https://www.intel.com.br/content/www/br/pt/research/neuromorphic-computing-loihi-2-technology-brief.html
- 🔬 IBM TrueNorth: https://research.ibm.com/truenorth
- 🔬 SpiNNaker: http://apt.cs.manchester.ac.uk/projects/SpiNNaker/

---

**Desenvolvido por:** Mauro Risonho de Paula Assumpção  
**Projeto:** Detecção de Fraude Neuromórfica  
**Data:** Dezembro 2025